In [1]:
import time,requests,json,random
import pandas as pd
import datetime as dt


In [23]:
class StockCrawling:
    req = None
    db = None

    def __init__(self):
        self.req = requests.session()

    def __del__(self):
        self.req.close()

    def show_realtime(self, *stock_id, stock_type):
        twse_url = 'http://mis.twse.com.tw/stock/api/getStockInfo.jsp'
        timestamp = int(time.time() * 1000)
        for target in stock_type:
            if(target=='上市'):
                channels = '|'.join('tse_{}.tw'.format(target_tse) for target_tse in stock_id)
            elif(target=='上櫃'):
                channels = '|'.join('otc_{}.tw'.format(target_tse) for target_tse in stock_id)
        query_url = '{}?&ex_ch={}&json=1&delay=0&_={}'.format(twse_url, channels, timestamp)

        headers = {'Accept-Language': 'zh-TW',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.81 Safari/537.36',
                   }

        self.req.get('http://mis.twse.com.tw/stock/index.jsp', headers=headers)
        response = self.req.get(query_url)
        if response.text.strip() == '':
            return None

        # d: 日期, h: 最高, l: 最低, c:代號, n: 名稱, t:時間, o: 開盤, v: 交易量, z: 成交價
        # load JSON from TSE
        #print(response.text)
        content = json.loads(response.text)
        # print (content)
        self.req.cookies.clear()
       
        return content['msgArray']

In [36]:
if __name__ == '__main__':
    # To ill stock IDs that you want.
    data = pd.read_csv("./stock_id_all.csv")
    stock_ids = [ str(i) for i in data['代號'].to_list() ]
    stock_type = [ str(i) for i in data['櫃別'].to_list() ]
    # print(stock_ids)
    # print(stock_type)
    crawling = StockCrawling()
    try:
        while True:
            now = dt.datetime.now()
            #if now < STOP_TIME:
            data = crawling.show_realtime(*stock_ids, stock_type=stock_type)
            records = []
            for s_dict in data:
                try:
                    
                    _dt = dt.datetime.strptime(s_dict['d'], "%Y%m%d").date().strftime('%Y-%m-%d')
                    _t = dt.datetime.strptime(s_dict['t'], '%H:%M:%S').time().strftime('%H:%M:%S')
                    # records.append([_dt, s_dict['c'], s_dict['n'],
                    #             float(s_dict['z']), float(s_dict['h']), float(s_dict['l']), float(s_dict['o']),
                    #             int(s_dict['v'])])
                    if(s_dict['z']!='-'):
                        s_dict['z'] = float(s_dict['z'])
                    else:
                        s_dict['z'] = float('NaN')
                    if(s_dict['tv']!='-'):
                        s_dict['tv'] = float(s_dict['tv'])
                    else:
                        s_dict['tv'] = float('NaN')
                    if(s_dict['h']!='-'):
                        s_dict['h'] = float(s_dict['h'])
                    else:
                        s_dict['h'] = float('NaN')
                    if(s_dict['l']!='-'):
                        s_dict['l'] = float(s_dict['l'])
                    else:
                        s_dict['l'] = float('NaN')
                    if(s_dict['o']!='-'):
                        s_dict['o'] = float(s_dict['o'])
                    else:
                        s_dict['o'] = float('NaN')
                    if(s_dict['v']!='-'):
                        s_dict['v'] = float(s_dict['v'])
                    else:
                        s_dict['v'] = float('NaN')

                    records.append([_dt, _t, s_dict['c'], s_dict['n'], s_dict['z'], s_dict['tv'],
                    s_dict['h'], s_dict['l'], float(s_dict['y']), float(s_dict['o']), int(s_dict['v'])])

                
                except ValueError as _e:
                    print("ValueErr : " + str(_e))
            print(records)
            _sleep_time = random.randint(2, 6)
                # sleep
            time.sleep(_sleep_time)
            print("====== " + time.strftime("%H:%M:%S") + " ======")
    finally:
        print("ERROR")
        del crawling



    #前167筆沒問題
    #第168筆開始會有問題，不知道是不是資料量太大
    #加油www
    
    # ┴┬┴┬／￣＼＿／￣＼
    # ┬┴┬┴▏　　▏▔▔▔▔＼
    # ┴┬┴／＼　／　　　　　　﹨
    # ┬┴∕　　　　　　　／　　　）
    # ┴┬▏　　　　　　　　●　　▏
    # ┬┴▏　　　　　　　　　　　▔█◤

    # ┴◢██◣　　　　　　 ＼＿＿／
    # ┬█████◣　　　　　　　／　　　　
    # ┴█████████████◣
    # ◢██████████████▆▄
    # █◤◢██◣◥█████████◤＼
    # ◥◢████　████████◤　　 ＼
    # ┴█████　██████◤　　　　　 ﹨

    # ┬│　　　│█████◤　　　　　　　　▏
    # ┴│　　　│　　　　　　　　　　　　　　▏
    # ┬∕　　　∕　　　　／▔▔▔＼　　　　 ∕
    # *∕＿＿_／﹨　　　∕　　　　　 ＼　　／＼
    # ┬┴┬┴┬┴＼ 　　 ＼_　　　　　﹨／　　﹨
    # ┴┬┴┬┴┬┴ ＼＿＿＿＼　　　　 ﹨／▔＼﹨／▔＼
    # ▲△▲▲╓╥╥╥╥╥╥╥╥＼　　 ∕　 ／▔﹨　／▔


ERROR


KeyError: 'msgArray'